In [ ]:
#!/usr/bin/env python
# coding: utf-8
import sys
sys.path.append('..')
import umbrella_mesh
import elastic_rods
import linkage_vis
from umbrella_mesh import UmbrellaEnergyType
from bending_validation import suppress_stdout as so
from visualization_helper import *

import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

from pipeline_helper import UmbrellaOptimizationCallback, allEnergies, allGradientNorms, allDesignObjectives, allDesignGradientNorms, set_joint_vector_field, show_center_joint_normal, show_joint_normal

from design_optimization_analysis import DesignOptimizationAnalysis

import umbrella_optimization
import umbrella_optimization_finite_diff
from umbrella_optimization import OptEnergyType

import numpy as np
import numpy.linalg as la

import pickle, gzip

from configuration import *

from datetime import datetime

import os

from load_jsondata import read_data, write_deformed_config
import mesh
import importlib, pipeline_helper


importlib.reload(pipeline_helper)

from pipeline_helper import set_joint_vector_field, show_center_joint_normal, show_joint_normal

from equilibrium_solve_analysis import EquilibriumSolveAnalysis
import py_newton_optimizer
OPTS = py_newton_optimizer.NewtonOptimizerOptions()
OPTS.gradTol = 1e-8
OPTS.verbose = 1
OPTS.beta = 1e-6
OPTS.niter = 300
OPTS.verboseNonPosDef = False

import time


import umbrella_optimization

import py_newton_optimizer
opt_opts = py_newton_optimizer.NewtonOptimizerOptions()
opt_opts.gradTol = 1e-8
opt_opts.verbose = 10
opt_opts.beta = 1e-6
opt_opts.niter = 600
opt_opts.verboseNonPosDef = False


import pipeline_helper, importlib, design_optimization_analysis
with so(): importlib.reload(pipeline_helper)
with so(): importlib.reload(design_optimization_analysis)

from pipeline_helper import UmbrellaOptimizationCallback

from umbrella_optimization import OptEnergyType

from design_optimization_analysis import DesignOptimizationAnalysis

import pickle 
import gzip

import compute_vibrational_modes

from configuration import *

In [ ]:
# ### Initialization
name = 'squid'
input_path = '../../data/{}.json.gz'.format(name)
io, input_data, target_mesh, curr_um, thickness, target_height_multiplier = parse_input(input_path)
# curr_um = pickle.load(gzip.open('../../Optimized_model/tigridia/Copy of tigridia_optimized_rest_state_2022_01_23_16_15_target_height_factor_5.0.pkl.gz'))
# curr_um = pickle.load(gzip.open('../../python/optimization_experiments/2022_01_26_10_14_saddle_5t/saddle_5t_force_equilibrium_2022_01_26_10_14_target_height_factor_5.0.pkl.gz', 'r'))
curr_um = pickle.load(gzip.open('saddle_5t_rest_state.pkl.gz'))

In [ ]:
# #### Pin Rigid Motion
# 
# 
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

In [ ]:
rod_colors = get_color_field(curr_um, input_data)

view = linkage_vis.LinkageViewer(curr_um, width=1024, height=512)
# set_surface_view_options(view, color = 'green', surface_color = 'gray', umbrella_transparent = False, surface_transparent = True)
view.averagedMaterialFrames = True
# view.showScalarField(rod_colors)
view.update(scalarField=rod_colors)
view.show()

In [ ]:
# np.save('{}_cam_view.npy'.format(name), view.getCameraParams())

In [ ]:
# view.setCameraParams(tuple([tuple(pt) for pt in np.load('{}_cam_view.npy'.format(name))]))

### Offscreen render

In [ ]:
import OffscreenRenderer

In [ ]:
width = 3840
height = 2160

In [ ]:
render = view.offscreenRenderer(width, height)

In [ ]:
def render_callback(prob, i):
    view.update(scalarField=rod_colors)
    geometry = view.getVisualizationGeometry()
    render.updateMeshData(geometry[0], geometry[2], rod_colors.colors())
    vw.writeFrame()

In [ ]:
def eqm_callback(prob, i):
    view.showScalarField(rod_colors)

### Undeployment

In [ ]:
# use_pin = False

# driver = curr_um.centralJoint()
# jdo = curr_um.dofOffsetForJoint(driver)
# fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()

# configure_umbrella_undeployment_step_one(curr_um, thickness, target_height_multiplier)

# OPTS.niter = 10

# results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
# results.success
# configure_umbrella_undeployment_step_two(curr_um)

# curr_um.attractionWeight = 0

# OPTS.niter = 100

# results = umbrella_mesh.compute_equilibrium(curr_um, callback = eqm_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)
# results.success

# pickle.dump(curr_um, gzip.open('{}_rest_state.pkl.gz'.format(name), 'w'))

### Deployment

In [ ]:
use_pin = False

driver = curr_um.centralJoint()
jdo = curr_um.dofOffsetForJoint(driver)
fixedVars = (list(range(jdo, jdo + 6)) if use_pin else []) + curr_um.rigidJointAngleDoFIndices()



In [ ]:
configure_umbrella_pre_deployment(curr_um, thickness, target_height_multiplier)


curr_um.attractionWeight = 1e-6
curr_um.scaleInputPosWeights(0.5, bdryMultiplier = 0.0)

In [ ]:
dw = np.zeros(curr_um.numUmbrellas())

In [ ]:
dw[10] = 10
dw[21] = 10

In [ ]:
curr_um.deploymentEnergyWeight = dw

In [ ]:
allGradientNorms(curr_um)

break_input_angle_symmetry(curr_um)

view.update(scalarField=rod_colors)

vw = OffscreenRenderer.video_writer.MeshRendererVideoWriter('{}_deployment.mp4'.format(name), render, outWidth = 1920, outHeight = 1080)

In [ ]:
break_input_angle_symmetry(curr_um)
results = umbrella_mesh.compute_equilibrium(curr_um, callback = render_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
dw[11] = 10
dw[20] = 10
curr_um.deploymentEnergyWeight = dw
results = umbrella_mesh.compute_equilibrium(curr_um, callback = render_callback, options = OPTS, fixedVars = fixedVars, elasticEnergyIncreaseFactorLimit=2.5)

In [ ]:
vw.finish()